In [3]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [4]:
hyper_epoch=100

In [ ]:
for _ in range(hyper_epoch):
    # model parameters
    reuse = np.random.choice([True,False])
    block = np.random.choice([NNConv_block,schnet_block])
    head = np.random.choice([cat3Head_type,swapHead_type,cat3HeadPool_type,set2setHead_type,SimpleHead_type])
    data = np.random.choice(['../Data/{}_data_ACSF_expand_PCA.pickle',\
                             '../Data/{}_data_SOAP_expand_PCA.pickle',\
                             '../Data/{}_data_atomInfo.pickle'])
    batch_size = 32
    dim = int(np.random.choice([128,256]))
    epochs = 50
    clip = 2
    layer1 = int(np.random.choice([2,3]))
    layer2 = int(np.random.choice([2,3]))
    factor = int(np.random.choice([2,3,4]))
    lr = 1e-4
    epochs_type = epochs

    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [reuse,block,head,data,batch_size,dim,clip,\
                                              layer1,layer2,factor,epochs]])))

    for i in range(8):
        # load type data
        data_type = data.split('.pickle')[0]+ '_type_'+str(i)+'.pickle'
        train_dl,val_dl = get_data(data_type,batch_size)

        # init model
        model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
        paras = trainable_parameter(model)
        opt = Adam(paras,lr=lr)
        scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)

        print('\nTraining type: {}'.format(i))
        model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,scheduler=scheduler)

        save_results(train_loss_perType,val_loss_perType,reuse,block,\
                     head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')

        save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')